In [1]:
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi

data = pd.read_csv('./data/film_loaction.CSV', encoding="CP949")
data = data[data['미디어타입'] == 'drama']
data.drop(['연번', '미디어타입', '장소타입', '휴무일', '최종작성일', '브레이크타임'], axis=1, inplace=True)

def trim_address(address):
    parts = address.split()
    if len(parts) >= 2:
        return ' '.join(parts[:2])
    return address

data['주소'] = data['주소'].apply(trim_address)
data.reset_index(drop=True, inplace=True)

# 주소 변환
def transform_address(address):
    first_part = address.split()[0]
    if len(first_part) == 4:
        return first_part
    else:
        return first_part[:2]

# 주소 변환 적용
data['주소'] = data['주소'].apply(transform_address)

len(data['주소'].unique())

# 크롤링 장소 이미지 데이터 추가
img = pd.read_csv('./data/data.CSV')
data['img'] = img['img']

17

In [4]:
data

,제목,장소명,장소설명,영업시간,주소,위도,경도,전화번호,img
0,(아는 건 별로 없지만) 가족입니다,커피파머,6회에서 김은희(한예리)와 안효석(이종원)이 윤태형(김태호)의 은신처에서 서울로 돌...,매일 09시 - 21시,경기,37.667523,126.72817,031-1899-8903,https://th.bing.com/th/id/OIP.FoMo-D1_gw_GUrsc...
1,(아는 건 별로 없지만) 가족입니다,셀렉토커피 남양주호평점,10회에서 하라(배윤경)는 은희(한예리)에게 이 커피숍에서 만나자고 메시지를 보낸다...,매일 08시 - 22시,경기,37.663558,127.24748,031-511-6362,https://th.bing.com/th/id/OIP.1Vl1Sv3JJOhZPmra...
2,(아는 건 별로 없지만) 가족입니다,카페 그루비,"안효석(이종원)은 바리스타로, 김은주(추자현)는 단골손님이다. 1화에서 이진숙(원미...",매일 12시 - 22시,경기,37.293983,127.05538,031-214-1288,https://th.bing.com/th/id/OIP.AfzMAYK2b8VZFz7v...
3,(아는 건 별로 없지만) 가족입니다,미리내 성지,시누이 도혜지(황우슬혜)는 남편이 가문의 회사를 상속받을 수 있도록 그녀가 길을 잃...,매일 09시 - 17시,경기,37.146102,127.25865,031-674-1256,https://th.bing.com/th/id/OIP.FKwVfi4SsgnhqTDP...
4,(아는 건 별로 없지만) 가족입니다,초은당,김은희(한예리)와 동료 서경옥(가득희)은 1회에서 작가 의뢰인을 만나기 위해 이 한...,매일 11시 - 20시,경기,37.611484,127.35397,070-4118-0770,https://th.bing.com/th/id/OIP.MZUfQAExanXbwLWg...
...,...,...,...,...,...,...,...,...,...
7334,You Raise Me Up (유 레이즈 미 업),서울마리나 클럽&요트,6회에서 산책 후 밤에 이 클럽 마당에 한강 기슭에 앉아 그들은 키스를 나눈다.,매일 10시 - 22시,서울,37.534829,126.91157,02-3780-8400,https://th.bing.com/th/id/OIP.Tmr9OpQ3vN1ZaMW1...
7335,You Raise Me Up (유 레이즈 미 업),한강대교,2회에서 절망적이고 우울한 도용식(윤시윤)이 이 다리에 와서 자신의 삶을 반성합니다.,매일 00시 - 24시,서울,37.517492,126.95897,02-2199-8580,https://th.bing.com/th/id/OIP.EtqJ0CViW6SlHxOQ...
7336,You Raise Me Up (유 레이즈 미 업),해밀톤 호텔별관,7회에서 도지혁(박기웅)이 이 술집에서 술을 마신다.,가게별 상이,서울,37.535099,126.99377,02-3786-6000,NaN
7337,You Raise Me Up (유 레이즈 미 업),북촌로5가길,8화에서 두라와 데이트를 하며 이 거리를 걷는다.,매일 00시 - 24시,서울,37.580327,126.98237,02-2148-1114,NaN


In [14]:
def generate_ngrams(text, n=2):
    tokens = text.split()
    ngrams = []
    for token in tokens:
        ngrams.extend([token[i:i+n] for i in range(len(token)-n+1)])
    return ngrams

# 예시 텍스트
text = "여기가 거기여"
ngrams = generate_ngrams(text, 2)
print(ngrams)

['여기', '기가', '거기', '기여']


In [13]:
# 2-gram 토큰화 함수
def generate_ngrams(text, n=2):
    tokens = text.split()
    ngrams = []
    for token in tokens:
        ngrams.extend([token[i:i+n] for i in range(len(token)-n+1)])
    return ngrams

# 촬영지 데이터 토큰화
tokenized_title = [generate_ngrams(doc, 2) for doc in data['제목']]
tokenized_place = [generate_ngrams(doc, 2) for doc in data['장소명']]
tokenized_address = [generate_ngrams(doc, 2) for doc in data['주소']]

# BM25 모델 토큰화 데이터로 초기화
# 드라마제목검색기
bm25_title = BM25Okapi(tokenized_title)
# 장소명검색기
bm25_place = BM25Okapi(tokenized_place)
# 지역명검색기
bm25_address = BM25Okapi(tokenized_address)


In [7]:
# 검색 함수
def search_result(query, s_option):
    # 사용자 검색 쿼리를 2-gram으로 토큰화
    tokenized_query = generate_ngrams(query, 2)
    
    # s_option에 따라 타이틀, 장소 또는 주소에 대해 BM25 점수 계산
    if s_option == 0:
        # 타이틀에 대해 BM25 점수 계산
        bm25_scores = bm25_title.get_scores(tokenized_query)
    elif s_option == 1:
        # 장소에 대해 BM25 점수 계산
        bm25_scores = bm25_place.get_scores(tokenized_query)
    elif s_option == 2:
        # 주소에 대해 BM25 점수 계산
        bm25_scores = bm25_address.get_scores(tokenized_query)
    
    print("BM25 Scores:", bm25_scores)  # 점수를 출력하여 확인합니다

    # 점수를 내림차순으로 정렬하여 상위 결과 인덱스 추출
    results = np.argsort(bm25_scores)[::-1]

    # 상위 결과 추출 및 점수 필터링
    top_n = 200  # 상위 200개의 결과만 고려
    # 상위 결과 중 점수가 3.5 보다 큰 결과만 필터링
    if s_option == 2:
        filtered_results = [idx for idx in results[:top_n] if bm25_scores[idx] > 0]
    else:
        filtered_results = [idx for idx in results[:top_n] if bm25_scores[idx] > 3.5]
    print("Filtered Results:", filtered_results)  # 필터링된 결과를 출력하여 확인합니다
    
    # 결과 데이터프레임 생성
    if len(filtered_results) > 0:
        results_df = data.iloc[filtered_results].copy()  # 필터링된 결과를 데이터프레임으로 변환
        results_df['index'] = filtered_results  # 인덱스 컬럼 추가
        return results_df.to_dict(orient='records')  # 결과를 딕셔너리 리스트로 반환
    else:
        return None  # 결과가 없을 경우 null 반환


In [11]:
search_result('서울',2)

BM25 Scores: [0.         0.         0.         ... 1.12046068 1.12046068 1.12046068]
Filtered Results: [7338, 1389, 4059, 4060, 1331, 1330, 1329, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4078, 1320, 1319, 4058, 4057, 4056, 4045, 4039, 1341, 4041, 4042, 4043, 4044, 4046, 4055, 1340, 1339, 4049, 4052, 4053, 4054, 1318, 4088, 4089, 1298, 4114, 4115, 4116, 4117, 4118, 1299, 1297, 4112, 1296, 1287, 1286, 4133, 4134, 4135, 4113, 4111, 1317, 1310, 1316, 4092, 1314, 1313, 1312, 1311, 1305, 4110, 1304, 1303, 1302, 1301, 1300, 4109, 4038, 1342, 4029, 1367, 3973, 3974, 1371, 1370, 1369, 1368, 1366, 3971, 3984, 3985, 3986, 3987, 3988, 3989, 3972, 3970, 3991, 1380, 1387, 1386, 3953, 3954, 1382, 1381, 1379, 3969, 3963, 3964, 3965, 3966, 3967, 3968, 3990, 3992, 4028, 4020, 1353, 1352, 1351, 4017, 4018, 4019, 4021, 1355, 4022, 4023, 4024, 4025, 4026, 4027, 1354, 1356, 3993, 1360, 3994, 1365, 1364, 1363, 1362, 1361, 1359, 4008, 1358, 4003, 4004, 4005, 4006, 4007, 4136, 4137, 4138, 4270, 4264, 4265, 42

[{'제목': 'You Raise Me Up (유 레이즈 미 업)',
  '장소명': '청계천',
  '장소설명': '1화에서 절망에 빠진 도용식(윤시윤)이 이 광장을 향해 걸어가며 자신의 삶을 성공한 사람들과 비교한다.',
  '영업시간': '매일 00시 - 24시',
  '주소': '서울',
  '위도': 37.569205,
  '경도': 126.97791,
  '전화번호': '02-2290-6114',
  'img': 'https://th.bing.com/th/id/OIP.6OF_cg8xhHqdUuLiYOcqrAHaE7?w=244&h=180&c=7&r=0&o=5&dpr=1.3&pid=1.7',
  'index': 7338},
 {'제목': '내조의 여왕',
  '장소명': '자하주택',
  '장소설명': '천지애(김남주)의 집으로 나온다.',
  '영업시간': '매일 00시 - 24시',
  '주소': '서울',
  '위도': 37.600665,
  '경도': 126.96326,
  '전화번호': '02-2148-5092',
  'img': 'https://th.bing.com/th/id/OIP._RYJfCqeTbTSJgYcfusaAQHaE7?w=300&h=200&c=7&r=0&o=5&dpr=1.3&pid=1.7',
  'index': 1389},
 {'제목': '식샤를 합시다 2',
  '장소명': '순대실록',
  '장소설명': '극중 윤두준과 서현진이 방문했던 순대 전문점',
  '영업시간': '매일 09시 - 23시 30분',
  '주소': '서울',
  '위도': 37.583241,
  '경도': 127.003047,
  '전화번호': '02-742-5338',
  'img': 'https://th.bing.com/th/id/OIP.MNTlB-QmDM1SbRFIPiNrqwHaFj?w=279&h=209&c=7&r=0&o=5&dpr=1.3&pid=1.7',
  'index': 4059},
 {'제목': '식샤를 합시다 2',
  '장소명': '하림각

In [9]:
import json
import jsonpickle
from flask import Response

In [10]:
# !pip install jsonpickle

In [12]:
import threading
from flask import Flask, request, jsonify

app = Flask(__name__)
@app.route('/')
def home():
    return "Flask server is running"

@app.route('/search', methods=['GET'])
def search_data():
    try:
        # 검색옵션
        s_option = int(request.args.get('s_option'))
        # 사용자 검색쿼리
        query = request.args.get('search')
        # 검색함수 대입
        result = search_result(query, s_option)
        # json 변환 후 webServer 반환
        return Response(jsonpickle.encode(result, unpicklable=False), mimetype='application/json')
    except Exception as e:
        return Response(jsonpickle.encode({"error": str(e)}, unpicklable=False), mimetype='application/json')
def run_flask():
    app.run(port=5002, debug=True, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
127.0.0.1 - - [31/Jul/2024 19:02:29] "GET /search?s_option=2&search=인천 HTTP/1.1" 200 -


BM25 Scores: [0. 0. 0. ... 0. 0. 0.]
Filtered Results: [5437, 613, 2995, 2994, 6667, 6666, 6665, 6664, 6663, 2993, 2992, 2991, 2990, 2303, 4119, 4120, 4121, 632, 2986, 4856, 1430, 1429, 1428, 614, 612, 366, 1443, 3034, 3030, 3029, 6710, 6709, 586, 5746, 5748, 5749, 2285, 4090, 3016, 3015, 598, 6694, 6693, 599, 1450, 2293, 2294, 1444, 1427, 5782, 5783, 1426, 1373, 4198, 1372, 4203, 2912, 2911, 4804, 723, 6560, 6559, 6558, 6557, 724, 4799, 4227, 4228, 4229, 1344, 2884, 2883, 6529, 6577, 6578, 5835, 2966, 1425, 2975, 1424, 648, 1423, 1422, 1421, 2967, 4144, 2319, 5834, 2320, 4839, 2324, 4162, 1399, 4169, 2933, 2932, 4189, 572, 571, 3038, 6845, 1585, 1584, 1583, 3929, 423, 424, 3930, 3931, 5615, 5616, 6881, 5617, 6874, 6870, 6869, 1573, 1569, 2187, 2188, 2189, 1557, 3171, 3172, 3173, 5596, 3877, 3878, 3879, 3213, 3212, 3211, 3201, 3198, 3197, 5597, 3174, 1599, 398, 1598, 1597, 2160, 2161, 3177, 3176, 3175, 456, 6844, 3039, 6843, 6786, 6785, 1506, 1505, 6772, 4030, 4031, 532, 4032, 4033, 40

127.0.0.1 - - [31/Jul/2024 19:02:31] "GET /search?s_option=2&search=대전 HTTP/1.1" 200 -


BM25 Scores: [0. 0. 0. ... 0. 0. 0.]
Filtered Results: [266, 2618, 7016, 4671, 6445, 5234, 367, 3451, 5011, 6225, 6226, 77, 479, 2104, 2989, 1595, 2988, 757, 2518, 6339, 2519]


...

BM25 Scores: [0. 0. 0. ... 0. 0. 0.]
Filtered Results: [4305, 4306, 4307, 4308, 4309, 4310, 4311, 4312, 4313, 4304, 4303, 4503, 4508, 4506, 4505, 4504, 4507, 4502, 4500, 4499, 4494, 4495, 4496, 4497, 4498, 4501, 4318, 4316, 4317, 4319, 5466, 5467, 5465, 5464, 5463, 4299, 4298, 4315, 4314, 4302, 4300, 5958, 4301, 6053, 6057, 6056, 6054, 6055, 2502, 2501, 4647, 4644, 4651, 4650, 4649, 4648, 4646, 4645, 4652, 2891, 777, 779, 780, 778, 1307, 1298, 1297, 1296, 1295, 1292, 1293, 1306, 1305, 1304, 1303, 1302, 1301, 1294, 1300, 1290, 1299, 1291]


127.0.0.1 - - [01/Aug/2024 16:54:52] "GET /search?s_option=0&search=오징어게임 HTTP/1.1" 200 -


BM25 Scores: [0. 0. 0. ... 0. 0. 0.]
Filtered Results: [5033, 5036, 5029, 5030, 5031, 5032, 5034, 5035, 5037, 5027, 5038, 5039, 5040, 5041, 5042, 5044, 5028, 5043, 2096, 2099, 2097, 2098, 3031, 3032, 3033, 3034, 44, 45, 46, 47, 48, 49, 50, 42, 59, 51, 52, 53, 54, 55, 56, 43, 37, 41, 40, 39, 38, 58, 36, 35, 34, 60, 57, 3781, 4518, 4519, 4521, 4522, 4523, 4516, 4515, 4525, 4510, 4517, 4511, 4512, 4513, 4514, 4524, 4520]


127.0.0.1 - - [01/Aug/2024 16:55:11] "GET /search?s_option=2&search=광주 HTTP/1.1" 200 -


BM25 Scores: [0. 0. 0. ... 0. 0. 0.]
Filtered Results: [4532, 1909]
